# Lightweight Fine-Tuning Project


TODO: In this cell, describe your choices for each of the following

- PEFT technique:
- Model:
- Evaluation approach:
- Fine-tuning dataset:


## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.


### Load dataset Yelp reviews


#### **Dataset summary**
**Source Data:**  
The [Yelp reviews dataset](https://huggingface.co/datasets/Yelp/yelp_review_full) consists of reviews from Yelp. It is extracted from the Yelp Dataset Challenge 2015 data.

**Languages**  
The reviews were mainly written in english.

#### **Dataset Structure**

**Data Instances**
A typical data point, comprises of a text and the corresponding label.

**Data Fields**
The simplified configuration includes:

- `text`: The review texts are escaped using double quotes ("), and any internal double quote is escaped by 2 double quotes (""). New lines are escaped by a backslash followed with an "n" character, that is "\n".
- `labels`: Corresponds to the score associated with the review (between 1 and 5).


In [1]:
from datasets import load_dataset
# Load the train and test splits of the imdb dataset
ds = load_dataset("yelp_review_full")
ds

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 650000
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 50000
    })
})

### Pre-process datasets

In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_name = "nlptown/bert-base-multilingual-uncased-sentiment"

tokenizer = AutoTokenizer.from_pretrained(model_name)

def preprocess_function(examples):
    """Preprocess the go emotions dataset by returning tokenized examples."""
    return tokenizer(examples["text"], padding="max_length", truncation=True, return_tensors="pt")

tokenized_ds = {}
splits = ["train", "test"]
for split in splits:
    tokenized_ds[split] = ds[split].map(preprocess_function, batched=True)


# Show the first example of the tokenized training set
print(tokenized_ds["train"][0])

{'label': 4, 'text': "dr. goldberg offers everything i look for in a general practitioner.  he's nice and easy to talk to without being patronizing; he's always on time in seeing his patients; he's affiliated with a top-notch hospital (nyu) which my parents have explained to me is very important in case something happens and you need surgery; and you can get referrals to see specialists without having to see him first.  really, what more do you need?  i'm sitting here trying to think of any complaints i have about him, but i'm really drawing a blank.", 'input_ids': [101, 11407, 119, 55144, 23829, 23225, 151, 18480, 10139, 10104, 143, 10619, 49411, 46172, 37241, 119, 10191, 112, 161, 24242, 10110, 24806, 10114, 20220, 10114, 13208, 11352, 27102, 41098, 132, 10191, 112, 161, 17503, 10125, 10573, 10104, 43842, 10235, 24946, 132, 10191, 112, 161, 46866, 10171, 143, 11397, 118, 10497, 10277, 13532, 113, 94146, 114, 10359, 11153, 17163, 10574, 40958, 10114, 10525, 10127, 12495, 12652, 10104,

In [7]:
tokenized_ds

{'train': Dataset({
     features: ['label', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
     num_rows: 650000
 }),
 'test': Dataset({
     features: ['label', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
     num_rows: 50000
 })}

In [8]:
tokenized_ds['train'].features

{'label': ClassLabel(names=['1 star', '2 star', '3 stars', '4 stars', '5 stars'], id=None),
 'text': Value(dtype='string', id=None),
 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
 'token_type_ids': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None),
 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None)}

### Load pre-trained model

In [5]:
model = AutoModelForSequenceClassification.from_pretrained(model_name)

In [5]:
print(model)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(105879, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1

Compute metrics

In [3]:
import evaluate
import numpy as np

metric = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

Create training arguments and trainer

In [10]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="bert_peft_trainer",
    learning_rate=1e-3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="steps",
    save_strategy= 'steps',
    push_to_hub= True,
    resume_from_checkpoint=True,
    load_best_model_at_end=True)

bert_peft_trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["test"],
    compute_metrics=compute_metrics
)


/home/daniel/miniconda3/envs/transformers_env/lib/python3.9/site-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


## Evaluate pre-trained model

In [16]:
pre_trained_results = bert_peft_trainer.evaluate()

In [17]:
#import pickle

# Save pre-trained results
#with open('pre_trained_results.pkl', 'wb') as pickle_file:
#    pickle.dump(pre_trained_results, pickle_file)

In [7]:
import pickle

# Load pre-trained results
with open('pre_trained_results.pkl', 'rb') as pickle_file:
    pre_trained_results = pickle.load(pickle_file)


print(pre_trained_results)

{'eval_loss': 0.9804139733314514, 'eval_accuracy': 0.57678, 'eval_runtime': 715.3828, 'eval_samples_per_second': 69.893, 'eval_steps_per_second': 4.368}


## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.


### Create PEFT

- `task_type`: the task to train for (sequence-to-sequence language modeling in this case)
- `inference_mode`: whether you’re using the model for inference or not
- `r`: the dimension of the low-rank matrices
- `lora_alpha`: the scaling factor for the low-rank matrices
- `lora_dropout`: the dropout probability of the LoRA layers

In [6]:
from peft import LoraConfig, get_peft_model, TaskType
# Define LoRA Config
lora_config = LoraConfig(
 r=16,
 lora_alpha=32,
 target_modules=["query", "value"],
 lora_dropout=0.05,
 bias="none",
 task_type=TaskType.SEQ_CLS, # this is necessary
 inference_mode=True
)

# add LoRA adaptor
model = get_peft_model(model, lora_config)
model.print_trainable_parameters() # see % trainable parameters

trainable params: 3,845 || all params: 167,953,930 || trainable%: 0.0023


Login in Huggingface Hub

In [8]:
from huggingface_hub import login
login()

Create training arguments and trainer

In [9]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="bert_peft_trainer",
    learning_rate=1e-3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="steps",
    eval_steps=10000,
    save_steps=10000,
    save_strategy= 'steps',
    push_to_hub= True,
    resume_from_checkpoint=True,
    load_best_model_at_end=True)

bert_peft_trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["test"],
    compute_metrics=compute_metrics
)


/home/daniel/miniconda3/envs/transformers_env/lib/python3.9/site-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Train model

In [ ]:
bert_peft_trainer.train(resume_from_checkpoint=True)

Evaluate model

In [ ]:
fine_tune_results = bert_peft_trainer.evaluate()

In [15]:
import pickle
with open('fine_tune_results.pkl', 'wb') as pickle_file:
    pickle.dump(fine_tune_results, pickle_file)
fine_tune_results

{'eval_loss': 0.9588460326194763, 'eval_accuracy': 0.58842}

Save model

In [17]:
model.save_pretrained("bert-peft-yelp")

In [75]:
bert_peft_trainer.push_to_hub()

adapter_model.safetensors:   0%|          | 0.00/2.38M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/DanielFarfan/bert_peft_trainer/commit/0105bd1f3d01b28c9625097fe0f48c5605851b29', commit_message='End of training', commit_description='', oid='0105bd1f3d01b28c9625097fe0f48c5605851b29', pr_url=None, pr_revision=None, pr_num=None)

## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.


Load original model

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_name = "nlptown/bert-base-multilingual-uncased-sentiment"

tokenizer = AutoTokenizer.from_pretrained(model_name)

Metrics for evaluation and trainer

In [9]:
from transformers import TrainingArguments, Trainer
import evaluate
import numpy as np

metric = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

training_args = TrainingArguments(output_dir="bert_peft_trainer",
    learning_rate=1e-3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="steps",
    save_strategy= 'steps',
    push_to_hub= True,
    resume_from_checkpoint=True,
    load_best_model_at_end=True)

original_model_trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["test"],
    compute_metrics=compute_metrics
)


/home/daniel/miniconda3/envs/transformers_env/lib/python3.9/site-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Evaluate original model

In [ ]:
pre_trained_evaluation = original_model_trainer.evaluate()
pre_trained_evaluation

Load PEFT model

In [6]:
from peft import AutoPeftModelForSequenceClassification
peft_model = AutoPeftModelForSequenceClassification.from_pretrained("bert-peft-yelp")

In [21]:
from transformers import TrainingArguments, Trainer
peft_trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["test"],
    compute_metrics=compute_metrics
)

In [22]:
fine_tune_results = peft_trainer.evaluate()
with open('fine_tune_results.pkl', 'wb') as pickle_file:
    pickle.dump(fine_tune_results, pickle_file)

In [1]:
import pickle
# Load pre-trained results
with open('pre_trained_results.pkl', 'rb') as pickle_file:
    pre_trained_results = pickle.load(pickle_file)

# Load pre-trained results
with open('fine_tune_results.pkl', 'rb') as pickle_file:
    fine_tune_results = pickle.load(pickle_file)

# Function to print formatted results
def print_results(title, results):
    print(f"{title}:")
    for key, value in results.items():
        print(f"  {key.replace('_', ' ').title()}: {value:.4f}" if isinstance(value, float) else f"  {key.replace('_', ' ').title()}: {value}")
    print("\n")

# Print the results
print_results("Pre-Trained Model Evaluation Results", pre_trained_results)
print_results("Fine-Tuned Model Evaluation Results", fine_tune_results)

Pre-Trained Model Evaluation Results:
  Eval Loss: 0.9804
  Eval Accuracy: 0.5768
  Eval Runtime: 715.3828
  Eval Samples Per Second: 69.8930
  Eval Steps Per Second: 4.3680


Fine-Tuned Model Evaluation Results:
  Eval Loss: 0.9588
  Eval Accuracy: 0.5884
  Eval Runtime: 756.5725
  Eval Samples Per Second: 66.0880
  Eval Steps Per Second: 4.1300




# Conclusion

Fine-tuning the model on the specific dataset has resulted in a notable improvement in both accuracy and loss, suggesting that the fine-tuned model is better suited for the specific task at hand. The increase in evaluation accuracy by approximately 1.16% and the decrease in evaluation loss by 0.06 demonstrate the effectiveness of the fine-tuning process.

These results highlight the importance of fine-tuning pre-trained models to adapt them to specific tasks or datasets, as it can lead to improved performance and more accurate predictions.